### Downloaded csv for house popular vote from https://docs.google.com/spreadsheet/ccc?key=0AjYj9mXElO_QdHVsbnNNdXRoaUE5QThHclNWaTgzb2c&usp=drive_web#gid=0

In [ ]:
# import csv
import json
# import numpy as np
import pandas as pd

In [ ]:
#dictionary between FIPS and states
with open("../data/st2Fips.json","rb") as f3:
    fips = json.load(f3)

#dictionary between full state names and abbreviations
with open("../data/state2St.json","rb") as f4:
    states = json.load(f4)

# read into pandas data frame
df = pd.read_csv('../data/114_2014_house_popular_vote.csv', header=0, skiprows=[0,1], usecols = [0,1,2,3,4,5,6,7], names = ['state','cd','pvi','name','party','demVotes','repVotes','othVotes'])
nRows = len(df)

df["state"] = df["state"].map(str.strip)
df["cd"] = df["cd"].map(str.strip)
df["pvi"] = df["pvi"].map(str.strip)
df["name"] = df["name"].map(str.strip)
df["party"] = df["party"].map(str.strip)

# set "At Large" districts to 0
df['cd'][df['cd'] == 'AL'] = '0'

# convert votes to numbers
df['demVotes'] = df['demVotes'].str.replace(',','').convert_objects(convert_numeric=True)
df['repVotes'] = df['repVotes'].str.replace(',','').convert_objects(convert_numeric=True)
df['othVotes'] = df['othVotes'].str.replace(',','').convert_objects(convert_numeric=True)


In [ ]:
thisState = ''
thisDistrict = ''

c114_dict = {}

for i in range(len(df)):
    if df['state'][i] != thisState:
        thisState = df['state'][i]
        thisStateAbbrev = states[thisState]
        # found a new state, start a new temporary dictionary
        stateDict = {}
        stateDict['state'] = thisStateAbbrev
        # initialize district party tally
        stateDict['demDistr'] = 0
        stateDict['repDistr'] = 0
        stateDict['othDistr'] = 0
        # initialize vote tally
        stateDict['demVotes'] = 0
        stateDict['repVotes'] = 0
        stateDict['othVotes'] = 0
    
    if thisState in states:
        cd = df['cd'][i]
        if len(cd) == 1:
            cd = '0' + cd
        
        newDistrict = False
        if cd != thisDistrict:
            thisDistrict = cd
            newDistrict = True
            # found a new district, start a new temporary dictionary
            cdDict = {}
            
            # initialize vote tally
            cdDict['repVotes'] = 0
            cdDict['demVotes'] = 0
            cdDict['othVotes'] = 0

        if df['party'][i] == 'D':
            thisParty = 'D'
            stateDict['demDistr'] += 1
        elif df['party'][i] == 'R':
            thisParty = 'R'
            stateDict['repDistr'] += 1
        else:
            thisParty = 'O'
            stateDict['othDistr'] += 1
        
        cdDict['name'] = df['name'][i].decode("ascii", "ignore")
        cdDict['party'] = thisParty

        if newDistrict or cd == thisDistrict:
            # tally dem, rep, and other votes
            cdDict['demVotes'] += df['demVotes'][i]
            stateDict['demVotes'] += df['demVotes'][i]
            cdDict['repVotes'] += df['repVotes'][i]
            stateDict['repVotes'] += df['repVotes'][i]
            cdDict['othVotes'] += df['othVotes'][i]
            stateDict['othVotes'] += df['othVotes'][i]
            stateDict[cd] = cdDict
        
        if ((i+1) < len(df) and thisState != df['state'][i+1]) or i+1 == len(df):
            c114_dict[fips[thisStateAbbrev]] = stateDict


In [ ]:
with open('../data/114_2014_house_popular_vote.json', 'wb') as f:
    f.write(json.dumps(c114_dict))